In [1]:
import rasterio
import numpy as np
import os
from PIL import Image
import json

def load_config(config_file='config.json'):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

def convert_tiff_to_jpg_satellite(
    input_tif_path, 
    output_jpg_path, 
    band_order=(1, 2, 3),
    min_percent=2,    # Lower cut for stretch
    max_percent=98,   # Upper cut for stretch
    black_threshold=10  # Brightness threshold below which pixels are considered black
):
    """
    Convert a multi-band satellite TIFF to an 8-bit RGB JPEG and crop out large black areas.
    - band_order: Which bands to use for R, G, B (1-based).
    - min_percent, max_percent: The percentile cuts for min/max stretching.
    - black_threshold: Pixels below this average intensity will be considered black (for cropping).
    """

    with rasterio.open(input_tif_path) as src:
        # Read the specified bands into numpy arrays
        arrs = [src.read(b).astype(np.float32) for b in band_order]
        # Stack into shape (height, width, 3)
        rgb = np.dstack(arrs)
        
        # Remove NaN or no-data (if any)
        mask_valid = ~np.isnan(rgb)
        # If we have NaNs, set them to 0 (you can adjust to your preference)
        rgb[~mask_valid] = 0
        
        # Optionally stretch the data for display
        # 1) Flatten to get global stats across all 3 channels
        flat = rgb.flatten()
        # We already handled NaNs above, so just compute percentiles
        min_val = np.percentile(flat, min_percent)
        max_val = np.percentile(flat, max_percent)

        # 2) Clip, then scale to [0..255]
        rgb = np.clip(rgb, min_val, max_val)
        rgb = ((rgb - min_val) / (max_val - min_val + 1e-6)) * 255.0
        
        # Convert to uint8
        rgb_8 = rgb.astype(np.uint8)
        
        # --------------------------------------------------------------------
        # Detect black/empty areas:
        # --------------------------------------------------------------------
        # Compute a brightness map per pixel (mean or max across channels).
        # Then consider anything below `black_threshold` to be "black".
        brightness = rgb_8.mean(axis=2)  # shape: (height, width)
        # If you'd rather be strict about any channel, you could do:
        # brightness = rgb_8.max(axis=2)
        
        # Create a mask for "non-black" pixels
        non_black_mask = brightness > black_threshold
        
        # Find the bounding box of the non-black region
        coords = np.argwhere(non_black_mask)
        if coords.size > 0:
            y_min, x_min = coords.min(axis=0)
            y_max, x_max = coords.max(axis=0)
            
            # Crop the RGB array
            cropped_rgb_8 = rgb_8[y_min:y_max+1, x_min:x_max+1, :]
        else:
            # Fallback: if everything is black, just keep the original
            # or choose to skip saving
            cropped_rgb_8 = rgb_8
        
        # Save to JPEG using Pillow
        img_cropped = Image.fromarray(cropped_rgb_8, mode='RGB')
        img_cropped.save(output_jpg_path, 'JPEG')

        print(f"Saved cropped JPEG to {output_jpg_path}")
        
def batch_tiff_to_jpg_satellite(
    input_dir, 
    output_dir, 
    band_order=(1, 2, 3)
):
    """
    Loop over a directory of TIFFs and convert each to JPEG,
    picking the desired band_order and using default stretch params.
    """
    os.makedirs(output_dir, exist_ok=True)
    for fname in os.listdir(input_dir):
        if fname.lower().endswith(('.tif', '.tiff')):
            in_path = os.path.join(input_dir, fname)
            base = os.path.splitext(fname)[0]
            out_path = os.path.join(output_dir, f"{base}.jpg")
            convert_tiff_to_jpg_satellite(
                in_path, out_path, band_order=band_order
            )

In [2]:
config = load_config()
batch_tiff_to_jpg_satellite(config['test_image_location'],config['cropped_jpg_location'])

Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000000_post_disaster.jpg
Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000009_pre_disaster.jpg
Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000009_post_disaster.jpg
Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000002_pre_disaster.jpg
Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000000_pre_disaster.jpg
Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000001_pre_disaster.jpg
Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000001_post_disaster.jpg
Saved cropped JPEG to data/test/cropped_jpg/malawi-cyclone_00000002_post_disaster.jpg
